In [3]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math

# 셀레늄
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# for문으로 크롤링 

In [4]:
# 검색어 입력
search = '삼성전자'

# 크롬 옵션
options = webdriver.ChromeOptions()

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=1000,1200") # window-size -> 기본 : 1920,1080

# import chromedriver_autoinstaller

# chrome_path = chromedriver_autoinstaller.install()
# driver = webdriver.Chrome(chrome_path, options=options)
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

time.sleep(0.01)
driver.get("https://www.naver.com") # -> 네이버 크롬 창이 뜬다.

# 네이버 검색어 입력 후 검색
element = driver.find_element("name", "query") 
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(search)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(0.01)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/krc/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.134/chromedriver] found in cache


In [5]:
# 뉴스 클릭
driver.find_element("link text", "뉴스").click() 
time.sleep(0.01)

In [6]:
# '오래된 순' 클릭
driver.find_element("link text", "오래된순").click()  
time.sleep(0.01)

In [48]:
# 뉴스 데이터 수집
# news_list = []
# date_list = []
# title_list = []

for i in range(400):    

    # 언론사
    newses = driver.find_elements('css selector', 'a.info.press')
    temp_list = []
    for news in newses:
        temp_list.append(news.text)
        temp_list = [x for x in temp_list ]
    news_list = news_list + temp_list 
    
    # 날짜
    dates = driver.find_elements('css selector', 'span.info')
    temp_list = []
    for date in dates:
        temp_list.append(date.text)
        temp_list = [x for x in temp_list ]
    date_list = date_list + temp_list 
    
    # 제목
    titles = driver.find_elements('css selector', 'a.news_tit')
    temp_list = []
    for title in titles:
        temp_list.append(title.text)
        temp_list = [x for x in temp_list ]
    title_list = title_list + temp_list 
    
    # 다음페이지 클릭
    driver.find_element('css selector', 'a.btn_next').click()
    time.sleep(0.001)

In [49]:
print(len(title_list))
print(news_list[-2:])
print(date_list[-2:])
print(title_list[-2:])

84000
['한국경제', '파이낸셜뉴스']
['2016.05.24.', '2016.05.24.']
['[명품 펀드 명품 자산운용사] 미래에셋자산운용, 연 평균 수익률 7% 이상…고...', '영종도 운서역 초역세권에 생활형숙박시설 ‘영종도 삼성홈큐브’ 분양']


In [55]:
splited_date = date_list[-1].split('.')
year = int(splited_date[0])
month = int(splited_date[1])
day = int(splited_date[2])
print(year, month, day)

2016 5 24


In [50]:
# df로 만들기
df = pd.DataFrame(list(zip(news_list, date_list, title_list)), columns = ['언론사','날짜', '제목'])
df

,언론사,날짜,제목
0,아주경제,2016.01.01.,"[그레이트 코리아] 한-중 FTA 발효 직후, 변화될 양국 교역 변화"
1,아주경제,2016.01.01.,[위기를 기회로] 주력산업의 활로 이렇게 뚫어라
2,아주경제,2016.01.01.,[2016 산업기상도] ‘비 오기 전 우산 챙긴다’… 기업들 불황극복에 ‘안간힘...
3,아주경제,2016.01.01.,[위기를 기회로]블루오션은 어디…차세대 먹거리 찾아라
4,아주경제,2016.01.01.,"[그레이트 코리아] 中 대륙 곳간, '한류·품질·온라인'으로 두드려라"
...,...,...,...
83995,비즈니스포스트,2016.05.07.,"삼성SDI, 중국에서 태양광 소재사업 확대에 역점"
83996,뉴스토마토,2016.05.07.,24일 코스피 순매매 기관 상위종목(확정)
83997,뉴스토마토,2016.05.07.,24일 코스피 순매매 외국인 상위종목(확정)
83998,한국경제,2016.05.07.,"[명품 펀드 명품 자산운용사] 미래에셋자산운용, 연 평균 수익률 7% 이상…고..."


In [51]:
# csv로 뽑기
df.to_csv('2016-05-07.csv')

In [52]:
driver.close()